# Basics of Compression
In this notebook, we will look at the basics of compression. This notebook is mostly aimed at giving you an understanding of concepts such as entropy, and various encoding methods.

In the applied compression notebook, we will look at some methods for lossless and lossy compression.

## Section 0. Preparing the Notebook
We start by importing the necessary libraries and then loading a sample image to work on.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# importing necessary packages
import numpy as np
from numpy import random
import cv2 as cv
from matplotlib import pyplot as plt
import bitarray as bt
from solutions import *
from utils import *
from typing import Tuple, Dict, List

In [ ]:
# loading the sample image and setting the colormap for pyplot
image = cv.imread('data/baboon.bmp', cv.IMREAD_GRAYSCALE)
plt.set_cmap('Greys_r')
_ = plt.imshow(image), plt.axis('off')

## Section 1. Encoding Redundency
To understand how changing our method of encoding might help us, let us do a little thought experiment first. We begin by drawing the histogram of the image.

### Section 1.1. Introduction to Huffman Coding

In [ ]:
drawHist(image)

As you can see, nearly half of the bins are unused. Normally we use 8-bit codewords for each intensity level, to keep the 256 possible intensity levels. But here we can *almost* get away with using 7-bit codewords for almost every bin. Let's say that we want to manually implement such a coding scheme. One way would be to take the 127 first possible codewords and assign to them the 127 most frequent intensity levels. The last codeword, i.e. $1111111$, will be split into the two codewords $11111110$ and $11111111$ and assigned to the two least frequent intensity levels, giving us codewords for all the 129 non-zero bins in the image.

Using such a coding scheme for writing data is pretty straightforward; you can simply replace each intensity level with its corresponding codeword. For interpreting the generated datastream as an image, we can simply start by reading the stream, 7-bit at a time. If we reach a septet equal to $1111111$, we can simply add the next bit to it, and interpret the resulting 8-bit codeword.

Try and create such a code for the baboon image. Use the *bitarray* library to generate the bitcode, and compare its length to the original bitcode.

You can read more about the *bitarray* library [here](https://pypi.org/project/bitarray/).

In [ ]:
H, W = image.shape
byte_array = image.flatten()

# ====== YOUR CODE ======

bitarray_compressed = None
bitarray_compressed_reference = baboonCompress(image)

# Comparing the bit lengths of the different representations
print(f'Bit length of the original image: {image.size * 8}')
print(f'Bit length of your compressed image: {len(bitarray_compressed)}')
print(f'Bit length of the reference compressed image: {len(bitarray_compressed_reference)}')

As you can see, using a different encoding resulted in a considerable reduction in the size of the image, without damaging the contents in any way. You can still recreate the original image using the provided bitcode, if you have the encoding dictionary. Of course, storing the dictionary along with the image will increase the size of the file, but with images that are sufficiently large, or have relatively high coding redundency, the size of this dictionary will be much lower than the decrease in size caused by our method of encoding.

We can expand our naive method to further reduce the necessary bit length for representing an image. One way of achieving this is by Huffman encoding. Below is a simple method of creating a Huffman encoding for a 5-symbol piece of data.

Imagine that we have a string of data, composed of the 5 symbols *A*, *B*, *C*, *D*, and *E*. The prevalence of these 5 symbols is as follows:

$
p_A = 0.33,
p_B = 0.33,
p_C = 0.17,
p_D = 0.09,
p_E = 0.08
$

We can start by creating a combined symbol *DE*, composed of the two least-probable symbols in our tables, and with a prevalence of *0.17*. Each time that we encounter a *D* or an *E* in our data, we put the encoding value for *DE* in the bit-array, followed by a *0* if the original symbol was a *D*, and a *1* if it was an *E*. We repeat this process with the two least probable symbols to create a combined symbol *CDE* with the probability 0.34. We continue this operation until we arrive at a combined symbol *ABCDE*. This symbol can be decoded according to the following graph.

![A representation of the proposed Huffman code](./figures/huffman_code.png)

The equivalent codeword for each symbol is the string of bits that mark its path from the tree's root, e.g. the codeword for A is *00*, D is *110*, etc. For decoding, we simply start traversing the tree according to the received bitcode, until we reach a leaf node. Therefore, *0001110* can be decoded as *ABD*.

This method is known as Huffman coding, and can be used to decrease the data size when there is a imbalance in the distribution of the source symbols. Here, we encode an image which uses most of the pixel intensities, to show that even then, a different method of encoding can be helpful in decreasing the size of data.

In [ ]:
# loading the new image and
image = cv.imread('data/barbara.bmp', cv.IMREAD_GRAYSCALE)
_ = plt.imshow(image), plt.axis('off')

The *bitarray* library itself has built-in functions for Huffman coding, which you can see in action below.

In [ ]:
from bitarray.util import canonical_huffman, canonical_decode

# Extracting the frequencies for the sample image
symbols = list(range(256))
frequencies = (np.histogram(image, bins=256, range=(-.5, 255.5))[0]).tolist()

# Creating a Huffman encoding/decoding dictionary
huffman_dict, count, symbol_canonical = canonical_huffman({i : frequencies[i] for i in range(256)})

# Encoding the image data
image_encoded = bt.bitarray()
for pixel in image.flatten():
    image_encoded += huffman_dict[pixel]

# Comparing the length of the encoded and raw data
print(f'Bit length of the raw image: {image.size * 8}')
print(f'Bit length of the encoded image: {len(image_encoded)}')

In [ ]:
# Decoding the bitstring
decoded_image = canonical_decode(image_encoded, count, symbol_canonical)
decoded_image = np.array(list(decoded_image), dtype=np.uint8).reshape(image.shape)
_ = plt.imshow(decoded_image), plt.axis('off'), plt.title('Decoded Image')

### Section 1.2. What is Entropy?
Entropy, as it is defined in information theory, can be described as the amount of *'information'* that a piece of data (string of bits, scribbles on a piece of paper, or what the weatherman says on TV) contains, and it represents a lower bound on how much we can compress the said data. A better understanding of entropy can be achieved by studying books on information theory, such as *Elements of Information Theory* by *Thomas M. Cover* and *Joy A. Thomas*.

For this notebook, we will try to avoid delving into the more complex side of information theory. But there is one formula that you ought to keep in mind, which is the basic formula for calculating the entropy in a piece of data, e.g. in a pixel. Let's say that in an image or a series of images, we count the number of times that each intensity value is represented, and create a probabilistic distribution function for all 255 intensity values.

$
\sum_{z = 0}^{255} p_{(z)} = 1
$

We can calculate the average entropy of a pixel with the following formula.

$
- \sum_{z = 0}^{255} p_{(z)} . log_2 p_{(z)}
$

You can try to calculate this for a variety of distributions. The highest entropy happens for the case where every pixel has about the same probability of appearing, where the average entropy will be 8 bits. For every other unbalanced distribution, the entropy will be lower than 8.

### Section 1.3. Entropy in Images
Note that the formula above assumes that every pixel's value is independent from the pixels around it, i.e. each pixel is generated randomly from a certain probability distribution. However, we know that this is not the case, since the value of pixels is often close to the pixels around them. Imagine that instead of saving the pixel intensities, we saved each pixel's difference with the pixel on its left. Doing so would certainly decrease the entropy greatly, since most of the values would be close to zero.

After doing this, we can then decode the image by saving the rightmost column of pixels, and creating every column from the difference map that we have. Try doing so below, and compare the achieved compression with the compression achieved by treating the pixel intensities as statistically independent. In the provided solution, the right column is not compressed.

In [ ]:
def diffCompress(image : np.ndarray) -> Tuple[bt.bitarray, int, Tuple[Dict, List, List]]:
    """
    Parameters:
    - image : np.ndarray
        A greyscale image, with dtype=np.uint8.
    Returns:
    - output : bt.bitarray
        The compressed version of the image, using Huffman coding
        and calculating the difference map.
    - height : int
        The height of the image, used to extract the rightmost
        column.
    - huffman : Tuple[Dict, List, List]
        The output of the canonical_huffman function.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Encoding the image
image_encoded_diff, height, huffman =  diffCompress(image)
image_encoded_diff_ref, height_ref, huffman_ref =  diffCompressRef(image)

# Comparing the sizes of raw and encoded images
print(f'Bit length of the raw image: {image.size * 8}')
print(f'Bit length of the encoded image, assuming independent pixel values): {len(image_encoded)}')
print(f'Bit length of the encoded image, by encoding difference values: {len(image_encoded_diff)}')
print(f'Bit length of the encoded image, by encoding difference values (reference): {len(image_encoded_diff_ref)}')

# Scratchpad
You can use this section to try out different codes, without making a mess of the notebook. :)

In [ ]:
image.size * 8